In [1]:
import pandas as pd
import numpy as np
import nltk
from scipy.sparse import coo_matrix
import re, json, string, datetime, random, itertools

from collections import OrderedDict, defaultdict

# sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import os

%load_ext lab_black
%matplotlib inline
# got an # This is an internal API and we don't need nested context for this. issues if you reloaded that

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

print(tf.version.VERSION)

2.2.0-rc2


In [2]:
# tf.test.gpu_device_name()

In [3]:
class GetStress:
    def __init__(self):

        # sys
        os.environ["TFHUB_CACHE_DIR"] = os.environ["SCRATCH"] + "/tmp/tfhub"
        ## path
        self.scratch_path = os.environ["SCRATCH"]
        self.input_folder = "covid-map/twitter-dataset-processed-2"
        self.input_path = os.path.join(self.scratch_path, self.input_folder)
        self.output_folder = "covid-map/twitter-dataset-processed-stress"

        # csv path list splited by month
        # self.tweets_filepath_set = self._read_dirs(self.input_path)
        # sample to test
        # self.sample_json_path = self.tweets_filepath_set["2020-01"][0]

        self.input_df_path = [
            os.path.join(self.input_path, x)
            for x in os.listdir(self.input_path)
            if x.endswith(".csv")
        ]

        self.model_path = os.path.join(
            self.scratch_path, "twitter-action/tfmodel/tweets_model2.h5"
        )
        self.embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
        self.model = keras.models.load_model(self.model_path)

    def _read_dirs(self, input_path):
        tweets_file_set = {}
        for month_folder in os.listdir(input_path):
            if month_folder.startswith("2020") and not month_folder.endswith(".zip"):
                tweets_file_set[month_folder] = []
                month_folder_path = os.path.join(input_path, month_folder)
                # print(month_folder_path)
                for tweets_file in os.listdir(month_folder_path):
                    if tweets_file.endswith("csv"):
                        tweets_file_path = os.path.join(month_folder_path, tweets_file)
                        tweets_file_set[month_folder].append(tweets_file_path)

        print("filepath:", tweets_file_set.keys())
        # print("all file count", sum([len(tweets_file_set[x]) for x in tweets_file_set]))
        return tweets_file_set

    def _get_cleaned_text_list_sample(self, df):
        """deprecated function"""
        embededText = self.embed(df["cleaned_text"])
        y_pred = model.predict(embededText, use_multiprocessing=True, batch_size=1024)

    def _get_stressful_index(self, text_list):
        return self.model.predict(
            self.embed(text_list), use_multiprocessing=True, batch_size=1024
        )

    def _write_df_windows(self, df_sample, df_path):
        output_path = df_path.replace(self.input_folder, self.output_folder)
        print("writing", output_path, df_sample.shape)
        df_sample.to_csv(output_path, index=False)

    def process_df(self, df_path):
        df_sample = pd.read_csv(df_path, lineterminator="\n")
        df_sample = df_sample[df_sample["cleaned_text"].notna()]
        # df_sample = df_sample[df_sample["CountyId"].notna()]
        print(df_sample.shape)

        if df_sample.shape[0] != 0:

            # predict text
            y_pred = self._get_stressful_index(df_sample["cleaned_text"].values)
            df_sample["stress_rate"] = y_pred

        self._write_df_windows(df_sample, df_path)
        del df_sample

    def process_df_windows(self, df_path_list):
        for df_path in df_path_list:
            print("reading", df_path)
            output_path = df_path.replace(self.input_folder, self.output_folder)
            if os.path.isfile(output_path):
                continue
            self.process_df(df_path)
        print("all done.")


GS = GetStress()
# GS.process_df_windows()

In [4]:
df_path_list = GS.input_df_path
GS.process_df_windows(df_path_list)

reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-03-31.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-02-25.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-03-16.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-02-05.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-02-10.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-04-10.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-02-20.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-01-21.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-03-06.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-03-21.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-04-05.csv
reading /scratch/user/diya.li/covid-map/twitter-dataset-processed-2/2020-01-31.csv
read

In [68]:
# test code below

def load_df_windows(input_df_path):
    for each_input_df_path in input_df_path:
        print("reading", each_input_df_path)
        load_df_samples(each_input_df_path)
    print("all sample done.")


def load_df_samples(sample_df_path):
    df_sample = pd.read_csv(sample_df_path)
    df_sample = df_sample[df_sample["cleaned_text"].notna()]
    df_sample = df_sample[df_sample["place_type"].notna()]
    print(df_sample.shape)

    # load model and get stressful text
    model = keras.models.load_model("tweets_model2.h5")
    y_pred = get_stressful_index(df_sample["cleaned_text"], model)
    df_sample["stress_rate"] = y_pred
    print(df_sample.shape, "done.")

    print("Start writing to", sample_df_path)
    write_df_windows(df_sample, sample_df_path)
    print("all done")


def get_stressful_index(text_list, model):
    y_pred = model.predict(embed(text_list), use_multiprocessing=True, batch_size=1024)
    return y_pred


def write_df_windows(df_sample, sample_df_path):
    df_sample.to_csv(output_path + sample_df_path, index=False)

In [2]:
# load_df_windows(input_df_path)

In [18]:
# # code below is trying to fix those conflict between 1 and 2
# import tensorflow.compat.v1 as tf

# tf.disable_v2_behavior()
# print(tf.version.VERSION)

# # check issues below for above code
# # https://stackoverflow.com/questions/57968999/runtimeerror-attempting-to-capture-an-eagertensor-without-building-a-function
# cfg = tf.compat.v1.ConfigProto(allow_soft_placement=True)
# cfg.gpu_options.allow_growth = True
# # version compat issues 1
# def get_text_embedding_compat_v1(text_list):
#     # tf.compat.v1.enable_eager_execution()
#     with tf.compat.v1.Session(config=cfg) as session:
#         session.run(
#             [
#                 tf.compat.v1.global_variables_initializer(),
#                 tf.compat.v1.tables_initializer(),
#             ]
#         )
#         return session.run(embed(text_list))


# # # version compat issues 2
# def get_text_embedding_v1(text_list):
#     with tf.Session(config=cfg) as session:
#         session.run(
#             [tf.global_variables_initializer(), tf.tables_initializer(),]
#         )
#         return session.run(embed(text_list))


# check is https://www.tensorflow.org/guide/effective_tf2
# to explain more about the migration from tf2

In [44]:
def getChunks(textList):
    chunksSize = 100000
    chkTimes = len(textList) // chunkSize
    chkLast = len(textList) - chkTimes * chunksSize
    cur = 0
    for chk in range(chkTimes):
        firstSlice = cur
        lastSlice = chunksSize * (cur + 1)
        embedText = textList[firstSlice:lastSlice]
        embededText = embed()


embededText = embed(df_sample["cleaned_text"])
# will crash runs on 16GB 4Core

In [17]:
# embededText = get_text_embedding_compat_v1()